# Lecture 13

We will be working with the original data of Chow and Siow (2006). First, let's load it up:

In [4]:
thepath = getwd()

nSingles= read.table(paste0(thepath, "/n_singles.txt"))
marr = read.table(paste0(thepath, "/marr.txt"))
nAvail = read.table(paste0(thepath, "/n_avail.txt"))

`nSingles` is a $60 \times 2$ vector of the number of singles in 1970. Each row is an age from $16 - 75$ and column $1$ is men and column $2$ is women.

`marr` is a $60 \times 60$ matrix of the number of matches between each men and women of different ages. For example entry $1,1$ are the number of matches between 16 year old men and 16 year old women.

We are only interested in the ages from $16-40$.

In [5]:
nbCateg = 25  # keep only the 16-40 yo population

muhat_x0 = nSingles[1:nbCateg, 1]
muhat_0y = nSingles[1:nbCateg, 2]
muhat_xy = as.matrix(marr[1:nbCateg, 1:nbCateg])
Nh = sum(muhat_xy) + sum(muhat_x0) + sum(muhat_0y) # Number of households

# Normalization
muhat_xy = muhat_xy/Nh
muhat_x0 = muhat_x0/Nh
muhat_0y = muhat_0y/Nh

# Marginals
n_x = apply(X = muhat_xy, MARGIN = 1, FUN = sum) + muhat_x0
m_y = apply(X = muhat_xy, MARGIN = 2, FUN = sum) + muhat_0y

nbX = nbY = nbCateg
xs = matrix(rep(1:nbX, nbY), nbX, nbY)
ys = matrix(rep(1:nbY, nbX), nbX, nbY, byrow = T)

We will form our $\Phi_{xy}$ such that
\begin{align*}
\Phi_{xy}^\lambda = \sum_{k} \lambda_k \phi_{xy}^k
\end{align*}
Let $\phi_{xy}^1 = -(x - y)^2$. Then $\phi_{xy}$ are the following basis functions
* $\phi_{xy} = \phi_{xy}^1 ((x + y)/2)^2$

* $\phi_{xy} = \phi_{xy}^1 ((x + y - 2)/2)^2$

* $\phi_{xy} = \phi_{xy}^1 (x + y - 1)^2$

In [6]:
# Form Phi matrix
phi1_xy = -c((xs - ys)^2)
phimat = cbind(phi1_xy, phi1_xy * c(((xs + ys)/2)^2), phi1_xy * c(((xs + ys - 2)/2)^2), 
    phi1_xy * c(((xs + ys - 1))^2))
nbK = dim(phimat)[2] # number of basis

# Standardize
phimat_mean = apply(X = phimat, MARGIN = 2, FUN = mean)
phimat_stdev = apply(X = phimat, MARGIN = 2, FUN = sd)
phimat = (phimat - matrix(phimat_mean, nbX * nbY, nbK, byrow = T))/matrix(phimat_stdev, 
    nbX * nbY, nbK, byrow = T)

Recall that our we are seeking to find the solution to the dual problem
\begin{align*}
\min_{u,v,\lambda}\left\{
\sum_{x}n_{x}u_{x}+\sum_{y}m_{y}v_{y}+2\sum_{xy}\sqrt{n_{x}m_{y}}\exp\left(
\frac{\Phi_{xy}^{\lambda}-u_{x}-v_{y}}{2}\right) 
+\sum_{x}n_{x}\exp\left(  -u_{x}\right)  +\sum_{y}m_{y}\exp\left(
-v_{y}\right)  -\sum_{xy}\hat{\mu}_{xy}\Phi_{xy}^{\lambda}%
\right\}.
\end{align*}
Let's set up our objective function

In [20]:
ObjFunc = function(uvlambda) {
    u_x = uvlambda[1:nbX]
    v_y = uvlambda[(nbX + 1):(nbX + nbY)]
    lambda = uvlambda[(nbX + nbY + 1):(nbX + nbY + nbK)]
    Phi_xy = matrix(phimat %*% matrix(lambda, ncol = 1), nbX, nbY)
    mu_xy = exp((Phi_xy - matrix(u_x, nbX, nbY) - matrix(v_y, nbX, nbY, byrow = T))/2)
    mu_x0 = exp(-u_x)
    mu_0y = exp(-v_y)
    
    val = sum(n_x * u_x) + sum(m_y * v_y) - sum(muhat_xy * Phi_xy) + 2 * sum(mu_xy) + 
        sum(mu_x0) + sum(mu_0y)
    grad_u = n_x - apply(X = mu_xy, MARGIN = 1, FUN = sum) - mu_x0
    grad_v = m_y - apply(X = mu_xy, MARGIN = 2, FUN = sum) - mu_0y
   
    grad_lambda = c(mu_xy - muhat_xy) %*% phimat
    grad = c(grad_u, grad_v, grad_lambda)
    res = val
    attr(res, "gradient") = grad
    # print(val) print(max(abs(uvlambda)))
    return(res)
}

And solve it using a non-linear solver

In [21]:
options(warn = -1)
outcome = nlm(ObjFunc, rep(0, nbX + nbY + nbK), iterlim = 1000)
uvlambdahat = outcome$estimate
lambdahat = uvlambdahat[(nbX + nbY + 1):(nbX + nbY + nbK)]
valuefunction = ObjFunc(uvlambdahat)
print(paste0("The value function is ", outcome$minimum))

[1] "The value function is 7.67702566726597"
